In [1]:
from random import randint

def generate_seq(length,n):
    return [randint(1,n-1) for i in range(length)]

print(generate_seq(4,10))

[2, 7, 2, 8]


In [2]:
sequence = generate_seq(5,50)
sequence

[41, 25, 12, 6, 1]

In [3]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(seq,n):
    enc = OneHotEncoder(n_values=n)
    return enc.fit_transform(np.array(seq).reshape(-1,1)).toarray()

In [4]:
one_hot_encode(sequence,50)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
     

In [5]:
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [6]:
one_hot_decode(one_hot_encode(sequence,50))

[41, 25, 12, 6, 1]

In [7]:
def get_pair(n_in,n_out,n):
    x = generate_seq(n_in,n)
    y = x[:n_out] + [0 for i in range(n_in-n_out)]
    x = one_hot_encode(x,n)
    y = one_hot_encode(y,n)
    x = x.reshape(1,x.shape[0],x.shape[1])
    y = y.reshape(1,y.shape[0],y.shape[1])
    return x,y

In [8]:
X,y = get_pair(5,2,20)
X.shape,y

((1, 5, 20),
 array([[[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.]]]))

In [9]:
one_hot_decode(X[0])

[5, 12, 8, 13, 14]

In [10]:
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2

In [13]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,TimeDistributed,RepeatVector

model = Sequential()
model.add(LSTM(150,input_shape = (n_timesteps_in,n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150,return_sequences=True))
model.add(TimeDistributed(Dense(n_features,activation='softmax')))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               120600    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 150)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 150)            180600    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 50)             7550      
Total params: 308,750
Trainable params: 308,750
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [63]:
for epoch in range(5000):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 2s - loss: 3.9075 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8890 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.9020 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8776 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8594 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8261 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8294 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8156 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7920 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7962 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7918 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7162 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6764 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5925 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5475 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5002 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3981 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3093 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.0797 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.8840 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.4701 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.3962 - acc: 0.6000
Ep

Epoch 1/1
 - 0s - loss: 1.6222 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5637 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6145 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5934 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5391 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6143 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7465 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6515 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6629 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6634 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5849 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5943 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6833 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6335 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6025 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5788 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5852 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5952 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6306 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6031 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6364 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5983 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.5973 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5645 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5755 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6525 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5848 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6703 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5712 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6397 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6122 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6156 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5955 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6043 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5295 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5927 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5936 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6682 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6454 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5914 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5978 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5697 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6008 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5682 - acc: 0.6000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.6174 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5644 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6060 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6340 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4784 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4432 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4787 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5414 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5444 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6396 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5362 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5936 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6248 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5259 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5887 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5190 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6338 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5464 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6231 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6169 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4980 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6087 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.3891 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4934 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.1269 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5744 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4645 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5515 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5405 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0283 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5323 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.0783 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.9521 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5416 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5095 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5654 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4395 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6034 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5363 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3358 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5476 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5803 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3944 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2719 - acc: 0.6000
Epoch 1/1
 - 0s - lo

...

Epoch 1/1
 - 0s - loss: 0.4376 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5615 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7305 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2273 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6448 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8160 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5033 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0508 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1897 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7873 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3856 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5598 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2638 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.4165 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8502 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1265 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9391 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6679 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3865 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7583 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4211 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2610 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.6724 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6084 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5817 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4585 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7054 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2318 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.8419 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2514 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6185 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1762 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1230 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4820 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4488 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5114 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5957 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8671 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5926 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5421 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4244 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7297 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4994 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8334 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.4837 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6566 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7139 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8613 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.4718 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4673 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2413 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7567 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9659 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6020 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.1585 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5636 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6374 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1060 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6526 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2712 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7645 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5307 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7086 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2679 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7086 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5780 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.7139 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4929 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4754 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5585 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7279 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7456 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7127 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4238 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5490 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5502 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2566 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5664 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7881 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1612 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1573 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7145 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2713 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0263 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5687 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3216 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8387 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1741 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.5534 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0106 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6156 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8782 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6092 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6257 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3095 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3834 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5694 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3979 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5903 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8338 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4876 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2707 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7314 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4519 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7387 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6454 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2936 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5125 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4021 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4695 - acc: 0.8000
Epoch 1/1


In [65]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    if np.array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Accuracy: 24.00%


# With Attention

In [11]:
from attention_decoder import AttentionDecoder

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(LSTM(150,input_shape=(n_timesteps_in,n_features),return_sequences=True))
model.add(AttentionDecoder(150,n_features))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [15]:
for epoch in range(5000):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 1s - loss: 3.9220 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8905 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8722 - acc: 0.2000
Epoch 1/1
 - 0s - loss: 3.8501 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8558 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8416 - acc: 0.4000
Epoch 1/1
 - 0s - loss: 3.8110 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7770 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7533 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7596 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7104 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6857 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6451 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6346 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5242 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5660 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4482 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3662 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3720 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.2918 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.1600 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.1003 - acc: 0.600

Epoch 1/1
 - 0s - loss: 1.6159 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5609 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5430 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5593 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5274 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5801 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4472 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5197 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4779 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5268 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5748 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6391 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5219 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6550 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5421 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5897 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5949 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5907 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4865 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5213 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4189 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4630 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.4893 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4750 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5331 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6006 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5083 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4522 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5486 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5816 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4398 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2556 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4665 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5449 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5358 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5018 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4488 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3439 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5468 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5876 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4158 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4906 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6111 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5240 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.4278 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3904 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4096 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4893 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3915 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6717 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2022 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2110 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1955 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2180 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3894 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1825 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5207 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3022 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1791 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2581 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0352 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.4877 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2013 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3716 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5148 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3959 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.5598 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9685 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0145 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3085 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.9671 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1125 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0135 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7366 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7940 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.3397 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6369 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0031 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7123 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3579 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.9637 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0006 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1248 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1330 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9829 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.5562 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5663 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0712 - acc: 0.8000
Epoch 1/1
 - 0s - lo

...

 - 0s - loss: 0.0206 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3708 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0223 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0185 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0230 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0234 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1759 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6483 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0034 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0065 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0019 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0690 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0204 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2314 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0314 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0049 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1170 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1396 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0860 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0104 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0633 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0313 - acc: 1.0000
Epoch 1/1
 - 0s - lo

 - 0s - loss: 0.0322 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1708 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1183 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1171 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6954 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1307 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0062 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1274 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0203 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0083 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0153 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1266 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0087 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0252 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0474 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0360 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0319 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1961 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0831 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0158 - acc: 1.0000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.0125 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0159 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7430 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2159 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0634 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0022 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0262 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0146 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0268 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1905 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0118 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0243 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0394 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0269 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3279 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0037 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0540 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0037 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0084 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1514 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1866 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 5.1560e-04 - acc: 1.0000
Epoch 

Epoch 1/1
 - 0s - loss: 0.0685 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0086 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0261 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0074 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0334 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0042 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0194 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0445 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2459 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0043 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0060 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1751 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0552 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0060 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1292 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1673 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0246 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0108 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0014 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0800 - acc: 1.0000
Epoch 1/1


Epoch 1/1
 - 0s - loss: 0.0541 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0074 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0051 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0140 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0067 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0116 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0047 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0042 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0947 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1194 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0098 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0033 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.2314 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0054 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0129 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1088 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0044 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0102 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0082 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0142 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0167 - acc: 1.0000
Epoch 1/1


In [17]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if np.array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Accuracy: 92.00%
Expected: [15, 11, 0, 0, 0] Predicted [15, 11, 0, 0, 0]
Expected: [17, 42, 0, 0, 0] Predicted [17, 42, 0, 0, 0]
Expected: [21, 15, 0, 0, 0] Predicted [21, 15, 0, 0, 0]
Expected: [21, 10, 0, 0, 0] Predicted [21, 10, 0, 0, 0]
Expected: [27, 40, 0, 0, 0] Predicted [27, 40, 0, 0, 0]
Expected: [45, 24, 0, 0, 0] Predicted [45, 24, 0, 0, 0]
Expected: [7, 2, 0, 0, 0] Predicted [7, 2, 0, 0, 0]
Expected: [33, 45, 0, 0, 0] Predicted [33, 45, 0, 0, 0]
Expected: [46, 18, 0, 0, 0] Predicted [46, 18, 0, 0, 0]
Expected: [47, 21, 0, 0, 0] Predicted [47, 21, 0, 0, 0]
